In [1]:
import json

In [2]:
with open('links.json', 'r') as f:
    gesamt = json.load(f)

In [4]:
with open('perfumes_data.json', 'r') as f:
    perfumes = json.load(f)
    partial = [element['url'] for element in perfumes]

In [5]:
gesamt

['https://www.parfumo.de/Parfums/Viktor_Rolf/Spicebomb',
 'https://www.parfumo.de/Parfums/Dior/Eau_Sauvage_Parfum',
 'https://www.parfumo.de/Parfums/mugler/AMen_Pure_Shot',
 'https://www.parfumo.de/Parfums/Dior/Dior_Homme_Sport_2012_Eau_de_Toilette',
 'https://www.parfumo.de/Parfums/Parfums_MDCI/Chypre_Palatin',
 'https://www.parfumo.de/Parfums/Guerlain/Homme_L_Eau_Boisee',
 'https://www.parfumo.de/Parfums/Joop/Joop_Homme_Wild',
 'https://www.parfumo.de/Parfums/Giorgio_Armani/Acqua_di_Gio_Essenza',
 'https://www.parfumo.de/Parfums/Giorgio_Armani/Armani_Code_Ultimate',
 'https://www.parfumo.de/Parfums/Cartier/Declaration_dun_Soir',
 'https://www.parfumo.de/Parfums/Chanel/Allure_Homme_Sport_Eau_Extrme',
 'https://www.parfumo.de/Parfums/Prada/Luna_Rossa_Eau_de_Toilette',
 'https://www.parfumo.de/Parfums/Amouage/Interlude_Man',
 'https://www.parfumo.de/Parfums/Tom_Ford/Noir_Eau_de_Parfum',
 'https://www.parfumo.de/Parfums/Yves_Saint_Laurent/La_Nuit_de_Homme_Frozen_Cologne',
 'https://www.p

In [6]:
partial

['https://www.parfumo.de/Parfums/Chloe/chloe-l-eau-de-parfum-intense',
 'https://www.parfumo.de/Parfums/Viktor_Rolf/flowerbomb-tiger-lily',
 'https://www.parfumo.de/Parfums/Elizabeth_Arden/green-tea-coconut-breeze',
 'https://www.parfumo.de/Parfums/Jean_Paul_Gaultier/gaultier-divine-le-parfum',
 'https://www.parfumo.de/Parfums/La_Rive/metaphor',
 'https://www.parfumo.de/Parfums/Rabanne/olympea-parfum',
 'https://www.parfumo.de/Parfums/Valentino/valentino-donna-born-in-roma-the-gold',
 'https://www.parfumo.de/Parfums/Clive_Christian/e-cashmere-musk',
 'https://www.parfumo.de/Parfums/Fragonard/lilas-2024',
 'https://www.parfumo.de/Parfums/Rabanne/million-gold-for-her',
 'https://www.parfumo.de/Parfums/Martine_Micallef/o-feminin',
 'https://www.parfumo.de/Parfums/ivy-lane/coconut-blush',
 'https://www.parfumo.de/Parfums/Marc_Jacobs/perfect-elixir',
 'https://www.parfumo.de/Parfums/Shiseido/ginza-night',
 'https://www.parfumo.de/Parfums/Giorgio_Armani/si-passione-eau-de-parfum-intense',
 '

In [7]:
gesamt = set(gesamt)
partial = set(partial)


In [9]:
diff = list(gesamt - partial)
len(diff)

1970

In [14]:
with open('diff_list.json', 'w') as f:
    json.dump(diff, f, indent=4)

In [15]:
perfumes

[{'name': "Chloé L'Eau de Parfum Intense",
  'brand': 'Chloé',
  'year': '2024',
  'detailed_ratings': {'DUFT': {'rating': '6.8', 'number_of_ratings': '100'},
   'HALTBARKEIT': {'rating': '7.6', 'number_of_ratings': '77'},
   'SILLAGE': {'rating': '7.2', 'number_of_ratings': '79'},
   'FLAKON': {'rating': '8.1', 'number_of_ratings': '77'},
   'PREIS-LEISTUNGS-VERHÄLTNIS': {'rating': '6.9', 'number_of_ratings': '57'}},
  'scent_strength': {'Blumig': 'l',
   'Süß': 'l',
   'Fruchtig': 'l',
   'Holzig': 'm',
   'Frisch': 's'},
  'top_notes': ['Himbeere'],
  'middle_notes': ['Rose'],
  'base_notes': ['Ambrox', 'Cashmeran', 'Zeder'],
  'all_notes': None,
  'ratings': {},
  'scent_types': {'Würzig': 7.0,
   'Synthetisch': 8.0,
   'Süß': 20.0,
   'Holzig': 15.0,
   'Fruchtig': 17.0,
   'Frisch': 9.0,
   'Blumig': 25.0,
   'Damen': 53.0,
   'Klassisch': 16.0,
   'Modern': 31.0,
   'Winter': 24.0,
   'Herbst': 31.0,
   'Sommer': 19.0,
   'Frühling': 26.0,
   'Täglich': 20.0,
   'Freizeit': 24.0

In [16]:
with open('perfumes.json', 'w') as f:
    json.dump(perfumes, f, indent=4)